# Projet SDP (Mention IA)

## Question 1 : Explication de type (1-1)

### Prologue
Nous nous plaçons dans l’exemple de comparaison entre deux candidats (ici notés `x` et `y`) évalués sur des cours $A\dots G$ avec des poids, et classés par **somme pondérée**.

### Données (x et y)  

On encode les notes suivantes (sur 100) et les poids :

- Critères : $A,B,C,D,E,F,G$
- Candidat $x$ : 85, 81, 71, 69, 75, 81, 88  
- Candidat $y$ : 81, 81, 75, 63, 67, 88, 95  
- Poids : 8, 7, 7, 6, 6, 5, 6

On définit la **contribution** de chaque critère $k$ à l’affirmation $x \succ y$ par : $d_k = w_k \,(x_k - y_k)$
- $d_k>0$ : critère **pour** $x\succ y$ (ensemble `pros`)  
- $d_k<0$ : critère **contre** $x\succ y$ (ensemble `cons`)  
- $d_k=0$ : critère **neutre**


In [71]:
# Données de l'exemple (x ≻ y)
criteres = ["A","B","C","D","E","F","G"]

notes_x = {"A":85,"B":81,"C":71,"D":69,"E":75,"F":81,"G":88}
notes_y = {"A":81,"B":81,"C":75,"D":63,"E":67,"F":88,"G":95}

poids = {"A":8,"B":7,"C":7,"D":6,"E":6,"F":5,"G":6}

# Contributions d_k = w_k (x_k - y_k)
d = {k: poids[k]*(notes_x[k]-notes_y[k]) for k in criteres}
d


{'A': 32, 'B': 0, 'C': -28, 'D': 36, 'E': 48, 'F': -35, 'G': -42}

### Formulation : ensembles pros/cons/neutres
On calcule automatiquement les ensembles `pros(x,y)`, `cons(x,y)` et `neutral(x,y)` à partir des contributions $d_k$.

In [72]:
pros = [k for k in criteres if d[k] > 0]
cons = [k for k in criteres if d[k] < 0]
neutral = [k for k in criteres if d[k] == 0]

print("Contributions d_k :")
for k in criteres:
    print(f"  {k} : {d[k]:+d}")

print("\npros(x,y)   =", pros)
print("cons(x,y)   =", cons)
print("neutral(x,y)=", neutral)


Contributions d_k :
  A : +32
  B : +0
  C : -28
  D : +36
  E : +48
  F : -35
  G : -42

pros(x,y)   = ['A', 'D', 'E']
cons(x,y)   = ['C', 'F', 'G']
neutral(x,y)= ['B']


### Trade-offs (1-1)

Un trade-off de type **(1-1)** est une paire $(P,C)$ avec :
- $P \in \mathrm{pros}(x,y)$
- $C \in \mathrm{cons}(x,y)$
- et $d_P + d_C > 0$

On peut donc lister toutes les paires admissibles $(P,C)$ par simple énumération.


In [73]:
tradeoffs_11 = []
for p in pros:
    for c in cons:
        if d[p] + d[c] > 0:
            tradeoffs_11.append((p,c,d[p]+d[c]))

print("Trade-offs (1-1) admissibles (P,C) tels que d_P + d_C > 0 :")
for (p,c,val) in sorted(tradeoffs_11, key=lambda t: (-t[2], t[0], t[1])):
    print(f"  ({p},{c}) : d_{p}+d_{c} = {val:+d}")

Trade-offs (1-1) admissibles (P,C) tels que d_P + d_C > 0 :
  (E,C) : d_E+d_C = +20
  (E,F) : d_E+d_F = +13
  (D,C) : d_D+d_C = +8
  (E,G) : d_E+d_G = +6
  (A,C) : d_A+d_C = +4
  (D,F) : d_D+d_F = +1


### Formulation par optimisation

#### Variables de décision
Pour chaque paire admissible $(p,c)$, on introduit une variable binaire :
$$
x_{p,c} \in \{0,1\}
$$
avec $x_{p,c}=1$ si on retient le trade-off $(p,c)$.

#### Contraintes
1) **Chaque critère contre** doit être compensé exactement une fois :
$$
\forall c \in \mathrm{cons}(x,y),\quad \sum_{p \in \mathrm{pros}(x,y)} x_{p,c} = 1
$$

2) **Disjonction** : un même critère pour ne peut servir qu’une fois :
$$
\forall p \in \mathrm{pros}(x,y),\quad \sum_{c \in \mathrm{cons}(x,y)} x_{p,c} \le 1
$$

3) Variables définies **uniquement** pour les paires admissibles $(p,c)$ où $d_p + d_c > 0$.

#### Objectif
On n'a rien à miniser, alors l'objectif sera $0$ (un problème de faisabilité).

#### Certificat de non-existence
Si le modèle est **infaisable**, on demande à Gurobi un **IIS** (*Irreducible Inconsistent Subsystem*) : un sous-ensemble minimal de contraintes incompatible. C’est un **certificat de non‑existence** exploitable dans un rendu.


In [74]:
# Implémentation avec Gurobi
# - Si le modèle est faisable : on extrait une explication (1-1)
# - Sinon : on calcule un IIS comme certificat de non-existence

import gurobipy as gp
from gurobipy import GRB

# Construction de l'ensemble des paires admissibles
def explication_1_1(d, pros, cons, neutral):
    A = [(p,c) for p in pros for c in cons if d[p] + d[c] > 0]

    m = gp.Model("explication_1_1")

    # Variables binaires x[p,c] pour (p,c) admissible
    xvar = m.addVars(A, vtype=GRB.BINARY, name="x")

    # Chaque 'contre' doit être couvert exactement une fois
    c_cover = {}
    for c in cons:
        c_cover[c] = m.addConstr(gp.quicksum(xvar[p,c] for p in pros if (p,c) in xvar) == 1,
                                 name=f"couvrir_cons[{c}]")

    # Chaque 'pour' utilisé au plus une fois (disjonction côté pros)
    p_use = {}
    for p in pros:
        p_use[p] = m.addConstr(gp.quicksum(xvar[p,c] for c in cons if (p,c) in xvar) <= 1,
                               name=f"utiliser_pros_au_plus_une_fois[{p}]")

    # Objectif : trouver une solution faisable
    m.setObjective(0, GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()

    if m.Status == GRB.OPTIMAL:
        print("\n Une explication (1-1) existe. Paires sélectionnées :\n")
        solution = [(p,c) for (p,c) in A if xvar[p,c].X > 0.5]
        for (p,c) in solution:
            print(f"  - ({p},{c}) : l'avantage de x sur y en {p}({d[p]:+d}) pèse plus fort que son désavantage en {c}({d[c]:+d})  => somme={d[p]+d[c]:+d}")
        for n in neutral:
            print(f"  - x et y se valent en {n}")
        print(f"\nLongueur l = {len(solution)}")


    elif m.Status == GRB.INFEASIBLE:
        print("\n Aucune explication (1-1) n'existe : modèle infaisable.")
        print("Calcul d'un IIS (certificat de non-existence) ...")
        m.computeIIS()
        print("\nContraintes appartenant à l'IIS :")
        for constr in m.getConstrs():
            if constr.IISConstr:
                print(" -", constr.ConstrName)
    else:
        print("\nStatut inattendu :", m.Status)

explication_1_1(d, pros, cons, neutral)


 Une explication (1-1) existe. Paires sélectionnées :

  - (A,C) : l'avantage de x sur y en A(+32) pèse plus fort que son désavantage en C(-28)  => somme=+4
  - (D,F) : l'avantage de x sur y en D(+36) pèse plus fort que son désavantage en F(-35)  => somme=+1
  - (E,G) : l'avantage de x sur y en E(+48) pèse plus fort que son désavantage en G(-42)  => somme=+6
  - x et y se valent en B

Longueur l = 3


## Question 2 : Explication de type (1-m)

### Données

In [75]:
criteres = ["A", "B", "C", "D", "E", "F", "G"]

notes = {
    "x":  {"A": 85, "B": 81, "C": 71, "D": 69, "E": 75, "F": 81, "G": 88},
    "y":  {"A": 81, "B": 81, "C": 75, "D": 63, "E": 67, "F": 88, "G": 95},
    "z":  {"A": 74, "B": 89, "C": 74, "D": 81, "E": 68, "F": 84, "G": 79},
    "t":  {"A": 74, "B": 71, "C": 84, "D": 91, "E": 77, "F": 76, "G": 73},
    "u":  {"A": 72, "B": 75, "C": 66, "D": 85, "E": 88, "F": 66, "G": 93},
    "v":  {"A": 71, "B": 73, "C": 63, "D": 92, "E": 76, "F": 79, "G": 93},
    "w":  {"A": 79, "B": 69, "C": 78, "D": 76, "E": 67, "F": 84, "G": 79},
    "w'": {"A": 57, "B": 76, "C": 81, "D": 76, "E": 82, "F": 86, "G": 77},
}

weights = {"A": 8, "B": 7, "C": 7, "D": 6, "E": 6, "F": 5, "G": 6}

### Comparaison $w \succ w'$

In [76]:
contributions = [weights[k] * (notes["w"][k] - notes["w'"][k]) for k in criteres]
contributions

[176, -49, -21, 0, -90, -10, 12]

On a plus de "contre" que de "pour", alors on ne peut pas utliser un trade-off type (1-1)

### Modélisation type (1-m)

#### 1. Variables de décision
Contrairement au cas (1-1), un argument *pour* peut maintenant couvrir plusieurs arguments *contre*.
Nous définissons une variable binaire qui associe un argument contre $c$ à un argument pour $p$ :

$$
x_{p,c} = 
\begin{cases} 
1 & \text{si l'argument } c \in \text{cons} \text{ est expliqué par } p \in \text{pros} \\
0 & \text{sinon}
\end{cases}
$$

Cette modélisation permet de construire implicitement des groupes $\mathcal{C}_p = \{c \mid x_{p,c} = 1\}$ associés à chaque $p$.

#### 2. Contraintes

**C1 : Partition des arguments "Contre" (Couverture)** Chaque argument négatif $c \in \text{cons}(x,y)$ doit appartenir à exactement un trade-off (il doit être expliqué par un seul $p$).
$$
\sum_{p \in \text{pros}} x_{p,c} = 1, \quad \forall c \in \text{cons}
$$

**C2 : Validité des Trade-offs (Positivité)** Pour chaque argument $p \in \text{pros}$, si celui-ci est utilisé pour expliquer un groupe de défauts, la somme des contributions doit être strictement positive.
$$
d_p + \sum_{c \in \text{cons}} d_c \cdot x_{p,c} \ge 0, \quad \forall p \in \text{pros}
$$
*Note : Si un argument $p$ n'est pas utilisé (tous les $x_{p,c}=0$), l'inégalité devient $d_p \ge 1$, ce qui est toujours vrai car $p \in \text{pros}$. Si $p$ est utilisé, cela garantit que l'avantage compense le cumul des désavantages.*

#### 3. Objectif
Comme précédemment, il s'agit d'un problème de satisfaction (trouver si une explication existe). La fonction objectif est constante ou nulle.
$$
\text{Minimiser } 0
$$

Par contre, par parcimonie, on peut minimiser le nombre de groupes formés:
$$
\text{Minimiser} \sum_{p \in \text{pros}} \sum_{c \in \text{cons}} x_{p,c}
$$

In [77]:
def explication_1_m(weights, notes, criteres, i1, i2):
    # Construction de l'ensemble des paires admissibles
    d = {k: weights[k]*(notes[i1][k]-notes[i2][k]) for k in criteres}
    pros = [k for k in criteres if d[k] > 0]
    cons = [k for k in criteres if d[k] < 0]
    neutral = [k for k in criteres if d[k] == 0]
    A = [(p,c) for p in pros for c in cons]

    m = gp.Model("explication_1_m")

    # Variables binaires x[p,c] pour (p,c) admissible
    xvar = m.addVars(A, vtype=GRB.BINARY, name="x")

    # Chaque 'contre' doit être couvert exactement une fois
    c_cover = {}
    for c in cons:
        c_cover[c] = m.addConstr(gp.quicksum(xvar[p,c] for p in pros if (p,c) in xvar) == 1,
                                 name=f"couvrir_cons[{c}]")

    # La contribution de chaque 'contre' doit être couverte au moins une fois
    # d[p] + sum_{c} x[p,c]*d[c] >= 0
    p_use = {}
    for p in pros:
        p_use[p] = m.addConstr(d[p] + gp.quicksum(d[c]*xvar[p,c] for c in cons if (p,c) in xvar) >= 0,
                               name=f"utiliser_pros_au_plus_une_fois[{p}]")

    # Objectif : minimiser le nombre de groupes utilisés
    m.setObjective(gp.quicksum(xvar[p,c] for (p,c) in A), GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()

    if m.Status == GRB.OPTIMAL:
        print("\n Une explication (1-m) existe. Groupes sélectionnés :\n")
        solution = [(p, c) for (p, c) in A if xvar[p, c].X > 0.5]

        # Regroupement par p
        groupes = {}
        for p, c in solution:
            groupes.setdefault(p, []).append(c)

        for p, cs in groupes.items():
            cs_list = ", ".join(cs)
            somme_c = sum(d[c] for c in cs)
            total = d[p] + somme_c
            print(
                f"  - ({p}, {{{cs_list}}}) : " +
                f"l'avantage en {p} ({d[p]:+d}) et désavantage en {{{cs_list}}} " +
                f"({somme_c:+d}) => somme={total:+d}"
            )

        for n in neutral:
            print(f"  - x et y se valent en {n}")

        print(f"\nLongueur l = {len(groupes)}")
    elif m.Status == GRB.INFEASIBLE:
        print("\n Aucune explication (1-m) n'existe : modèle infaisable.")
        print("Calcul d'un IIS (certificat de non-existence) ...")
        m.computeIIS()
        print("\nContraintes appartenant à l'IIS :")
        for constr in m.getConstrs():
            if constr.IISConstr:
                print(" -", constr.ConstrName)
    else:
        print("\nStatut inattendu :", m.Status)

explication_1_m(weights, notes, criteres, "w", "w'")


 Une explication (1-m) existe. Groupes sélectionnés :

  - (A, {B, C, E, F}) : l'avantage en A (+176) et désavantage en {B, C, E, F} (-170) => somme=+6
  - x et y se valent en D

Longueur l = 1


## Question 3 : Explication de type (m-1)

### Comparaison $u \succ v$

In [78]:
explication_1_m(weights, notes, criteres, "u", "v")


 Aucune explication (1-m) n'existe : modèle infaisable.
Calcul d'un IIS (certificat de non-existence) ...

Contraintes appartenant à l'IIS :
 - couvrir_cons[D]
 - couvrir_cons[F]
 - utiliser_pros_au_plus_une_fois[A]
 - utiliser_pros_au_plus_une_fois[B]
 - utiliser_pros_au_plus_une_fois[C]
 - utiliser_pros_au_plus_une_fois[E]


Le solveur n'a pas trouvé de solution qui satisfasse toutes les contraintes

### Modélisation type (m-1)

#### 1. Variables de décision
Similairement au cas (1-m), un argument *contre* peut maintenant couvrir plusieurs arguments *pour*.
Nous définissons une variable binaire qui associe un argument contre $c$ à un argument pour $p$ :

$$
x_{p,c} =   
\begin{cases} 
1 & \text{si l'argument } c \in \text{cons} \text{ est expliqué par } p \in \text{pros} \\
0 & \text{sinon}
\end{cases}
$$

Cette modélisation permet de construire implicitement des groupes $\mathcal{P}_c = \{p \mid x_{p,c} = 1\}$ associés à chaque $c$.

#### 2. Contraintes

**C1: Chaque contre doit être expliqué** : on impose une compensation strictement positive.
Comme les contributions sont entières (notes/poids entiers), $>0$ équivaut à $\ge 1$ :
$$
d_c + \sum_{p \in \mathrm{pros}} d_p \cdot x_{p,c} \ge 0 \quad \forall c\in\mathrm{cons}.
$$

**C2 : Un pro au plus une fois** (disjonction des ensembles de pros) :
$$
\sum_{c\in\mathrm{cons}} x_{p,c} \le 1 \quad \forall p\in\mathrm{pros}.
$$

(Optionnellement, on peut aussi imposer $\sum_{p} a_{p,c}\ge 1$, mais c’est déjà induit si $\mathrm{contrib}(c)<0$.)

#### 3. Objectif
Comme précédemment, il s'agit d'un problème de satisfaction (trouver si une explication existe). La fonction objectif est constante ou nulle.
$$
\text{Minimiser } 0
$$

Par contre, par parcimonie, on peut minimiser le nombre de groupes formés:
$$
\text{Minimiser} \sum_{p \in \text{pros}} \sum_{c \in \text{cons}} x_{p,c}
$$

In [79]:
def explication_m_1(weights, notes, criteres, i1, i2):
    # Construction de l'ensemble des paires admissibles
    d = {k: weights[k]*(notes[i1][k]-notes[i2][k]) for k in criteres}
    pros = [k for k in criteres if d[k] > 0]
    cons = [k for k in criteres if d[k] < 0]
    neutral = [k for k in criteres if d[k] == 0]

    A = [(p,c) for p in pros for c in cons]

    m = gp.Model("explication_m_1")

    # Variables binaires x[p,c] pour (p,c) admissible
    xvar = m.addVars(A, vtype=GRB.BINARY, name="x")

    # La contribution de chaque 'contre' doit être couverte au moins une fois
    # d[c] + sum_{p} x[p,c]*d[p] >= 0
    c_cover = {}
    for c in cons:
        c_cover[c] = m.addConstr(d[c] + gp.quicksum(d[p]*xvar[p,c] for p in pros if (p,c) in xvar) >= 0,
                                 name=f"couvrir_cons[{c}]")

    # Chaque 'pour' utilisé au plus une fois (disjonction côté pros)
    p_use = {}
    for p in pros:
        p_use[p] = m.addConstr(gp.quicksum(xvar[p,c] for c in cons if (p,c) in xvar) <= 1,
                               name=f"utiliser_pros_au_plus_une_fois[{p}]")

    # Objectif : minimiser le nombre de groupes utilisés
    m.setObjective(gp.quicksum(xvar[p,c] for (p,c) in A), GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()

    if m.Status == GRB.OPTIMAL:
        print("\n Une explication (m-1) existe. Groupes sélectionnés :\n")
        solution = [(p, c) for (p, c) in A if xvar[p, c].X > 0.5]

        # Regroupement par c
        groupes = {}
        for p, c in solution:
            groupes.setdefault(c, []).append(p)

        for c, ps in groupes.items():
            ps_list = ", ".join(ps)
            somme_p = sum(d[p] for p in ps)
            total = d[c] + somme_p
            print(
                f"  - ({{{ps_list}}}, {c}) : " +
                f"l'avantage en {c} ({d[c]:+d}) et désavantage en {{{ps_list}}} " +
                f"({somme_p:+d}) => somme={total:+d}"
            )

        for n in neutral:
            print(f"  - x et y se valent en {n}")

        print(f"\nLongueur l = {len(groupes)}")
    elif m.Status == GRB.INFEASIBLE:
        print("\n Aucune explication (m-1) n'existe : modèle infaisable.")
        print("Calcul d'un IIS (certificat de non-existence) ...")
        m.computeIIS()
        print("\nContraintes appartenant à l'IIS :")
        for constr in m.getConstrs():
            if constr.IISConstr:
                print(" -", constr.ConstrName)
    else:
        print("\nStatut inattendu :", m.Status)

explication_m_1(weights, notes, criteres, "u", "v")


 Une explication (m-1) existe. Groupes sélectionnés :

  - ({A, B, C}, D) : l'avantage en D (-42) et désavantage en {A, B, C} (+43) => somme=+1
  - ({E}, F) : l'avantage en F (-65) et désavantage en {E} (+72) => somme=+7
  - x et y se valent en G

Longueur l = 2


### Comparaison $y \succ z$

In [80]:
explication_m_1(weights, notes, criteres, "y", "z")


 Une explication (m-1) existe. Groupes sélectionnés :

  - ({A}, B) : l'avantage en B (-56) et désavantage en {A} (+56) => somme=+0
  - ({C}, E) : l'avantage en E (-6) et désavantage en {C} (+7) => somme=+1
  - ({F, G}, D) : l'avantage en D (-108) et désavantage en {F, G} (+116) => somme=+8

Longueur l = 3


## Question 4 : Explication de type (m-1) et (1-m)

### Comparaison $z \succ t$

In [81]:
explication_m_1(weights, notes, criteres, "z", "t")


 Aucune explication (m-1) n'existe : modèle infaisable.
Calcul d'un IIS (certificat de non-existence) ...

Contraintes appartenant à l'IIS :
 - couvrir_cons[C]
 - couvrir_cons[D]
 - couvrir_cons[E]
 - utiliser_pros_au_plus_une_fois[B]
 - utiliser_pros_au_plus_une_fois[G]


In [82]:
explication_1_m(weights, notes, criteres, "z", "t")


 Aucune explication (1-m) n'existe : modèle infaisable.
Calcul d'un IIS (certificat de non-existence) ...

Contraintes appartenant à l'IIS :
 - couvrir_cons[C]
 - couvrir_cons[D]
 - utiliser_pros_au_plus_une_fois[B]
 - utiliser_pros_au_plus_une_fois[F]
 - utiliser_pros_au_plus_une_fois[G]


### Modélisation de type (1-m) et (m-1) ensemble

* Soit $\mathcal{P}$ l'ensemble des critères pour de $x$ (Pros).
* Soit $\mathcal{C}$ l'ensemble des critères contre $x$ (Cons).
* Soit $d_k = w_k (x_k - y_k)$ la contribution pondérée du critère $k$.
    * Pour tout $p \in \mathcal{P}$, $d_p > 0$.
    * Pour tout $c \in \mathcal{C}$, $d_c < 0$.

### 1. Variables de Décision

**Variables de structure (liens) :**
* $x_{p,c} \in \{0,1\}$ : vaut 1 si le critère $p$ couvre le critère $c$ dans un trade-off de type **(1-m)**.
* $y_{p,c} \in \{0,1\}$ : vaut 1 si le critère $p$ aide à compenser le critère $c$ dans un trade-off de type **(m-1)**.

**Variables d'activation (rôles) :**
* $u_p \in \{0,1\}$ : vaut 1 si $p$ est la **tête** d'un trade-off (1-m).
* $v_c \in \{0,1\}$ : vaut 1 si $c$ est la **tête** d'un trade-off (m-1).

### 2. Contraintes

**C1 : Partitionnement des Contre (Couverture stricte)** Chaque critère *Contre* $c \in \mathcal{C}$ doit être traité exactement une fois. Il existe deux façons exclusives d'être traité :
1.  Soit il est couvert par un critère *Pour* dans une structure (1-m) (il est alors une feuille : $\exists p, x_{p,c}=1$).
2.  Soit il est la tête de son propre groupe (m-1) (il est alors une racine : $v_c = 1$).

$$
\sum_{p \in \mathcal{P}} x_{p,c} + v_c = 1 \quad \forall c \in \mathcal{C}
$$

**C2 : Disjonction des Pour (Utilisation unique)** Chaque critère *Pour* $p \in \mathcal{P}$ peut être utilisé au plus une fois. Il existe deux rôles possibles :
1.  Soit il est la tête d'un groupe (1-m) ($u_p = 1$).
2.  Soit il aide à couvrir un critère *Contre* dans un groupe (m-1) (il est alors une feuille : $\exists c, y_{p,c}=1$).

$$
u_p + \sum_{c \in \mathcal{C}} y_{p,c} \le 1 \quad \forall p \in \mathcal{P}
$$

**C3 : Couplage Logique** On ne peut activer un lien que si la tête correspondante est active.
* Si $p$ couvre $c$ en (1-m), alors $p$ doit être déclaré comme tête (1-m).
* Si $p$ aide $c$ en (m-1), alors $c$ doit être déclaré comme tête (m-1).

$$
x_{p,c} \le u_p \quad \forall p \in \mathcal{P}, \forall c \in \mathcal{C}
$$
$$
y_{p,c} \le v_c \quad \forall p \in \mathcal{P}, \forall c \in \mathcal{C}
$$

**C4 : Solvabilité des Trade-offs** Chaque groupe formé doit avoir une somme pondérée positive.

* **Pour les groupes (1-m) :** L'avantage de la tête $p$ doit être supérieur à la somme des désavantages des $c$ qu'il couvre.
$$
d_p \cdot u_p + \sum_{c \in \mathcal{C}} d_c \cdot x_{p,c} \ge 0 \quad \forall p \in \mathcal{P}
$$

* **Pour les groupes (m-1) :** La somme des avantages des $p$ aidants doit être supérieure au désavantage de la tête $c$.
$$
d_c \cdot v_c + \sum_{p \in \mathcal{P}} d_p \cdot y_{p,c} \ge 0 \quad \forall c \in \mathcal{C}
$$

#### 3. Objectif
Comme précédemment, il s'agit d'un problème de satisfaction (trouver si une explication existe). La fonction objectif est constante ou nulle.
$$
\text{Minimiser } 0
$$

Par contre, par parcimonie, on peut minimiser le nombre de groupes formés:
$$
\text{Minimiser} \sum_{p \in \text{pros}} \sum_{c \in \text{cons}} x_{p,c} + y_{p,c}
$$

In [ ]:
def explication_mixte(weights, notes, criteres, i1, i2):
    print(f"--- Recherche d'explication MIXTE (1-m) et (m-1) pour {i1} > {i2} ---")

    # Construction de l'ensemble des paires admissibles
    d = {k: weights[k]*(notes[i1][k]-notes[i2][k]) for k in criteres}
    pros = [k for k in criteres if d[k] > 0]
    cons = [k for k in criteres if d[k] < 0]
    neutral = [k for k in criteres if d[k] == 0]

    if sum(d.values()) <= 0:
        print(f"Erreur: {i1} n'est pas strictement meilleur que {i2} au global.")
        return

    m = gp.Model("explication_mixte")

    # Variables de décision
    # x[p,c] = 1 si p couvre c dans une structure (1-m)
    x = m.addVars([(p,c) for p in pros for c in cons], vtype=GRB.BINARY, name="x_1m")

    # y[p,c] = 1 si p aide c dans une structure (m-1)
    y = m.addVars([(p,c) for p in pros for c in cons], vtype=GRB.BINARY, name="y_m1")

    # u[p] = 1 si p est la TÊTE d'un groupe (1-m)
    u = m.addVars(pros, vtype=GRB.BINARY, name="u_head_1m")
    
    # v[c] = 1 si c est la TÊTE d'un groupe (m-1)
    v = m.addVars(cons, vtype=GRB.BINARY, name="v_head_m1")

    # Contraintes
    
    # C1: Chaque 'Contre' doit être couvert exactement une fois
    # Soit il est couvert par un x[p,c] (feuille d'un 1-m), soit il est tête v[c] (chef d'un m-1)
    for c in cons:
        m.addConstr(gp.quicksum(x[p,c] for p in pros) + v[c] == 1, name=f"cover_cons_{c}")

    # C2: Chaque 'Pour' est utilisé au plus une fois
    # Soit il est tête u[p] (chef d'un 1-m), soit il aide via y[p,c] (feuille d'un m-1)
    for p in pros:
        m.addConstr(u[p] + gp.quicksum(y[p,c] for c in cons) <= 1, name=f"use_pros_{p}")

    # C3: Liens logiques
    for p in pros:
        for c in cons:
            # Si x[p,c]=1, alors p doit être une tête (1-m)
            m.addConstr(x[p,c] <= u[p], name=f"link_x_u_{p}_{c}")
            # Si y[p,c]=1, alors c doit être une tête (m-1)
            m.addConstr(y[p,c] <= v[c], name=f"link_y_v_{p}_{c}")

    # C4: Validité des poids
    # Pour les groupes (1-m) : Avantage de p >= somme des désavantages c couverts
    for p in pros:
        # Note: d[c] est négatif, donc on additionne. Ex: 100 + (-30)*1 + (-40)*1 >= 0
        m.addConstr(d[p]*u[p] + gp.quicksum(d[c]*x[p,c] for c in cons) >= 0, name=f"budget_1m_{p}")

    # Pour les groupes (m-1) : Somme des avantages p aidants >= désavantage de c
    for c in cons:
        m.addConstr(d[c]*v[c] + gp.quicksum(d[p]*y[p,c] for p in pros) >= 0, name=f"budget_m1_{c}")

    # Objectif : minimiser le nombre de trade-offs utilisés
    # m.setObjective(gp.quicksum(u[p] for p in pros) + gp.quicksum(v[c] for c in cons), GRB.MINIMIZE)
    m.setObjective(0, GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()

    # 4. Affichage des résultats
    if m.Status == GRB.OPTIMAL:
        print("\nUne explication MIXTE existe. Groupes sélectionnés :\n")

        # Récupération et affichage des groupes (1-m)
        for p in pros:
            if u[p].X > 0.5:
                # p est une tête de (1-m), trouvons ses feuilles
                cs = [c for c in cons if x[p,c].X > 0.5]
                if cs:
                    cs_str = ", ".join(cs)
                    somme_c = sum(d[c] for c in cs)
                    print(f"  - [Type 1-m] ({p} vs {{{cs_str}}}) : "
                          f"l'avantage en {p} ({d[p]:.1f}) et les désavantages ({somme_c:.1f}) "
                          f"=> somme={d[p]+somme_c:.1f}")
        
        # Récupération et affichage des groupes (m-1)
        for c in cons:
            if v[c].X > 0.5:
                # c est une tête de (m-1), trouvons ses feuilles (pros qui l'aident)
                ps = [p for p in pros if y[p,c].X > 0.5]
                if ps:
                    ps_str = ", ".join(ps)
                    somme_p = sum(d[p] for p in ps)
                    print(f"  - [Type m-1] ({{{ps_str}}} vs {c}) : "
                          f"les avantages ({somme_p:.1f}) >= le désavantage {c} ({d[c]:.1f}) "
                          f"=> somme={d[c]+somme_p:.1f}")

        # Affichage des neutres
        if neutral:
            print(f"  - [Neutre]   Les candidats se valent sur : {', '.join(neutral)}")
            
    elif m.Status == GRB.INFEASIBLE:
        print("\nPas d'explication mixte possible.")
        m.computeIIS()
        print("\nContraintes conflictuelles (IIS) :")
        for constr in m.getConstrs():
            if constr.IISConstr:
                print(" -", constr.ConstrName)
    else:
        print("\nStatut inattendu :", m.Status)

explication_mixte(weights, notes, criteres, "z", "t")

--- Recherche d'explication MIXTE (1-m) et (m-1) pour z > t ---

Une explication MIXTE existe. Groupes sélectionnés :

  - [Type 1-m] (B vs {D, E}) : l'avantage en B (126.0) et les désavantages (-114.0) => somme=12.0
  - [Type m-1] ({F, G} vs C) : les avantages (76.0) >= le désavantage C (-70.0) => somme=6.0
  - [Neutre]   Les candidats se valent sur : A


In [84]:
criteres = ["A", "B", "C", "D", "E", "F", "G"]
weights = {"A": 8, "B": 7, "C": 7, "D": 6, "E": 6, "F": 5, "G": 6}

notes = {
    "a1": {"A": 89, "B": 74, "C": 81, "D": 68, "E": 84, "F": 79,   "G": 77},
    "a2": {"A": 71, "B": 84, "C": 91, "D": 79, "E": 78, "F": 73.5, "G": 77}
}

explication_mixte(weights, notes, criteres, "a1", "a2")

--- Recherche d'explication MIXTE (1-m) et (m-1) pour a1 > a2 ---

Pas d'explication mixte possible.

Contraintes conflictuelles (IIS) :
 - cover_cons_B
 - cover_cons_C
 - cover_cons_D
 - use_pros_A
 - budget_1m_A
 - budget_1m_E
 - budget_1m_F
 - budget_m1_B
 - budget_m1_C
 - budget_m1_D


Nous ne pouvons pas expliquer pourquoi « a2 » est moins bien classé que « a1 » uniquement à l'aide d'explications de type (1-m) et (m-1). Il faut une explication de type (m-n).

## Question 6 : Application sur Breast Cancer Wisconsin (Original)

Une fois les concepts théoriques établis, nous appliquons notre algorithme le plus robuste sur un jeu de données réel : **Breast Cancer Wisconsin**.

L'objectif est de transformer une "boîte noire" (le score de risque calculé par une Régression Logistique) en une explication intelligible pour un médecin, en utilisant des structures de type **(1-m)**, **(m-1)** ou **mixtes**.

### Méthodologie

1.  **Entraînement :** Nous utilisons `ucimlrepo` pour charger les données et `sklearn` pour entraîner une Régression Logistique. Cela nous fournit les **poids ($w$)** réels de chaque examen médical.
2.  **Recherche de Conflit :** Il est trivial d'expliquer pourquoi une patiente très malade est plus risquée qu'une patiente parfaitement saine. Le défi réside dans les **cas limites**.
3.  **Algorithme de Recherche :** Nous implémentons une boucle qui tire des paires de patientes au hasard jusqu'à trouver une situation de **"Trade-off"** :
    *   La patiente $A$ a un score de risque global plus élevé que $B$.
    *   MAIS, la patiente $B$ est meilleure que $A$ sur la majorité des critères, ou $A$ est meilleure que $B$ sur certains critères spécifiques.
    *   C'est la présence simultanée d'arguments **POUR** (aggravants) et **CONTRE** (rassurants) qui nécessite une explication complexe.

4.  **Résolution :** Nous appelons notre fonction générique `explication_mixte` (définie en Q4) pour structurer automatiquement l'argumentation.

In [85]:
from ucimlrepo import fetch_ucirepo
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# --- 1. Chargement et Préparation des Données ---
print("Chargement du dataset Breast Cancer Wisconsin...")
dataset = fetch_ucirepo(id=15) 
X_raw = dataset.data.features
y_raw = dataset.data.targets 

# Nettoyage : Suppression des NaN et de la colonne ID
full_data = pd.concat([X_raw, y_raw], axis=1).dropna()
if 'Sample_code_number' in full_data.columns:
    full_data = full_data.drop(columns=['Sample_code_number'])

target_col = y_raw.columns[0]
X = full_data.drop(columns=[target_col])
y = full_data[target_col] # 2 = Benign, 4 = Malignant

# Standardisation (Indispensable pour comparer les poids)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
features = X.columns.tolist()

# --- 2. Entraînement du Modèle (Boîte Noire) ---
clf = LogisticRegression(max_iter=5000)
clf.fit(X_scaled, y)

# Extraction des poids appris (w)
weights_lr = {feat: coef for feat, coef in zip(features, clf.coef_[0])}
print(f"Modèle entraîné sur {len(X)} patientes. Poids extraits pour {len(weights_lr)} critères.")

# --- 3. Recherche Automatique d'un Cas Pertinent ---
# On cherche une paire (Gagnant, Perdant) qui possède à la fois des Pros et des Cons

max_attempts = 2000
found_case = False
attempt = 0

print(f"\nRecherche d'un cas de conflit (Trade-off) en cours...")

while attempt < max_attempts and not found_case:
    attempt += 1
    
    # A. Tirage aléatoire
    idx1, idx2 = np.random.choice(range(len(X)), 2, replace=False)
    
    # B. Calcul des scores (Distance à l'hyperplan)
    s1 = clf.decision_function(X_scaled[idx1].reshape(1, -1))[0]
    s2 = clf.decision_function(X_scaled[idx2].reshape(1, -1))[0]
    
    # C. Identifier le plus risqué (Gagnant du classement)
    if s1 > s2:
        w_name, l_name = f"Patiente_{idx1}", f"Patiente_{idx2}"
        w_idx, l_idx = idx1, idx2
        sc_w, sc_l = s1, s2
    else:
        w_name, l_name = f"Patiente_{idx2}", f"Patiente_{idx1}"
        w_idx, l_idx = idx2, idx1
        sc_w, sc_l = s2, s1
        
    # D. Vérification des Pros/Cons
    # On construit le dictionnaire de notes nécessaire pour 'explication_mixte'
    notes_temp = {
        w_name: dict(zip(features, X_scaled[w_idx])),
        l_name: dict(zip(features, X_scaled[l_idx]))
    }
    
    # Calcul rapide des différences pondérées pour vérifier s'il y a conflit
    diffs = [weights_lr[k]*(notes_temp[w_name][k] - notes_temp[l_name][k]) for k in features]
    has_pros = any(d > 0.01 for d in diffs) # Marge de 0.01 pour éviter les erreurs d'arrondi
    has_cons = any(d < -0.01 for d in diffs)
    
    if has_pros and has_cons:
        found_case = True
        print(f"Comparaison : {w_name} (Score={sc_w:.2f}) > {l_name} (Score={sc_l:.2f})")
        print("-" * 80)
        
        # --- E. Génération de l'Explication ---
        # Utilisation de la fonction générique définie en Q4
        explication_mixte(weights_lr, notes_temp, features, w_name, l_name)

if not found_case:
    print("Aucun cas de conflit intéressant trouvé (dominance totale probable sur les tirages).")

Chargement du dataset Breast Cancer Wisconsin...
Modèle entraîné sur 683 patientes. Poids extraits pour 9 critères.

Recherche d'un cas de conflit (Trade-off) en cours...
Comparaison : Patiente_521 (Score=-3.92) > Patiente_341 (Score=-4.47)
--------------------------------------------------------------------------------
--- Recherche d'explication MIXTE (1-m) et (m-1) pour Patiente_521 > Patiente_341 ---

Une explication MIXTE existe. Groupes sélectionnés :

  - [Type m-1] ({Bland_chromatin} vs Single_epithelial_cell_size) : les avantages (0.4) >= le désavantage Single_epithelial_cell_size (-0.1) => somme=0.3
  - [Type m-1] ({Clump_thickness} vs Normal_nucleoli) : les avantages (0.5) >= le désavantage Normal_nucleoli (-0.2) => somme=0.3
  - [Neutre]   Les candidats se valent sur : Uniformity_of_cell_size, Uniformity_of_cell_shape, Marginal_adhesion, Bare_nuclei, Mitoses


### Interprétation des Résultats

L'exécution ci-dessus démontre la capacité de notre solveur (`explication_mixte`) à s'adapter à des données réelles continues.

Contrairement aux exemples théoriques :
1.  Les poids et les notes sont des **nombres flottants**.
2.  Les relations ne sont pas toujours pures (1-m) ou (m-1), mais souvent une combinaison.

L'affichage nous permet de dire au médecin : *"Bien que la patiente B ait des indicateurs X et Y plus rassurants (Cons), le risque global est dominé par l'indicateur Z de la patiente A (Pro de type 1-m) ou par la combinaison des indicateurs U et V (Pro de type m-1)."*

## Question 7 : 27 criteria

In [86]:
import pandas as pd
df = pd.read_excel("data27crit.xlsx", skiprows=4, index_col=0).dropna(how='all')
df.head()

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,A
weight,2.0,1.0,5.0,3.000000,4.0,6.000000,1.0,7.000000,4.0,3.000000,...,6.000000,2.0,7.0,1.0,2.0,6.0,3.0,5.0,5.0,1.0
solution 1,532.0,120.0,330.0,54.000000,12.0,125.000000,320.0,59.400000,125.0,325.000000,...,125.000000,577.0,231.0,677.0,32.0,197.0,123.0,224.0,324.1,53.0
solution 2,94.0,-756.0,174.4,-254.666667,225.5,162.333333,670.0,19.971429,127.5,37.666667,...,239.333333,155.0,349.0,363.0,206.0,294.0,201.0,214.8,142.9,465.0
solution 3,174.0,652.0,-1094.0,-176.000000,-854.0,-573.000000,6.0,50.600000,515.0,-395.000000,...,-479.000000,107.0,-37.0,-483.0,706.0,-301.0,27.0,490.0,-922.1,-717.0
solution 4,604.0,976.0,680.0,-52.000000,-234.0,333.000000,-254.0,-608.600000,-523.0,277.000000,...,927.000000,-295.0,769.0,-39.0,-192.0,-509.0,199.0,54.0,-393.9,617.0


In [87]:
criteres = df.columns.tolist()
weights = df.iloc[0].to_dict()
notes = df.drop(df.index[0]).T.to_dict()

In [88]:
explication_mixte(weights, notes, criteres, "solution 1", "solution 2")

--- Recherche d'explication MIXTE (1-m) et (m-1) pour solution 1 > solution 2 ---

Une explication MIXTE existe. Groupes sélectionnés :

  - [Type 1-m] (a vs {e}) : l'avantage en a (876.0) et les désavantages (-854.0) => somme=22.0
  - [Type 1-m] (j vs {r}) : l'avantage en j (862.0) et les désavantages (-686.0) => somme=176.0
  - [Type 1-m] (m vs {A}) : l'avantage en m (524.0) et les désavantages (-412.0) => somme=112.0
  - [Type 1-m] (n vs {i, p}) : l'avantage en n (968.0) et les désavantages (-910.0) => somme=58.0
  - [Type 1-m] (o vs {g}) : l'avantage en o (380.0) et les désavantages (-350.0) => somme=30.0
  - [Type 1-m] (u vs {l, x}) : l'avantage en u (314.0) et les désavantages (-302.0) => somme=12.0
  - [Type m-1] ({h} vs f) : les avantages (276.0) >= le désavantage f (-224.0) => somme=52.0
  - [Type m-1] ({b} vs k) : les avantages (876.0) >= le désavantage k (-496.0) => somme=380.0
  - [Type m-1] ({z} vs q) : les avantages (906.0) >= le désavantage q (-514.0) => somme=392.0
  - 

In [89]:
explication_mixte(weights, notes, criteres, "solution 1", "solution 3")

--- Recherche d'explication MIXTE (1-m) et (m-1) pour solution 1 > solution 3 ---

Une explication MIXTE existe. Groupes sélectionnés :

  - [Type m-1] ({c} vs b) : les avantages (7120.0) >= le désavantage b (-532.0) => somme=6588.0
  - [Type m-1] ({e} vs i) : les avantages (3464.0) >= le désavantage i (-1560.0) => somme=1904.0
  - [Type m-1] ({f} vs k) : les avantages (4188.0) >= le désavantage k (-2112.0) => somme=2076.0
  - [Type m-1] ({j} vs l) : les avantages (2160.0) >= le désavantage l (-634.0) => somme=1526.0
  - [Type m-1] ({m} vs p) : les avantages (2554.0) >= le désavantage p (-1536.0) => somme=1018.0
  - [Type m-1] ({n} vs v) : les avantages (2508.0) >= le désavantage v (-1348.0) => somme=1160.0
  - [Type m-1] ({a, d, o, q, r, t, w, z} vs y) : les avantages (18981.0) >= le désavantage y (-1330.0) => somme=17651.0


In [90]:
explication_mixte(weights, notes, criteres, "solution 1", "solution 4")

--- Recherche d'explication MIXTE (1-m) et (m-1) pour solution 1 > solution 4 ---

Une explication MIXTE existe. Groupes sélectionnés :

  - [Type 1-m] (d vs {x}) : l'avantage en d (318.0) et les désavantages (-228.0) => somme=90.0
  - [Type 1-m] (e vs {b}) : l'avantage en e (984.0) et les désavantages (-856.0) => somme=128.0
  - [Type 1-m] (h vs {c, p, q}) : l'avantage en h (4676.0) et les désavantages (-4216.0) => somme=460.0
  - [Type 1-m] (k vs {n, o, A}) : l'avantage en k (4560.0) et les désavantages (-2610.0) => somme=1950.0
  - [Type m-1] ({v} vs a) : les avantages (448.0) >= le désavantage a (-144.0) => somme=304.0
  - [Type m-1] ({s} vs f) : les avantages (1744.0) >= le désavantage f (-1248.0) => somme=496.0
  - [Type m-1] ({w} vs l) : les avantages (4236.0) >= le désavantage l (-3990.0) => somme=246.0
  - [Type m-1] ({g, i, j, u, y} vs r) : les avantages (4876.0) >= le désavantage r (-4812.0) => somme=64.0
  - [Type m-1] ({m, z} vs t) : les avantages (4042.0) >= le désavantag

In [91]:
explication_mixte(weights, notes, criteres, "solution 2", "solution 3")

--- Recherche d'explication MIXTE (1-m) et (m-1) pour solution 2 > solution 3 ---

Une explication MIXTE existe. Groupes sélectionnés :

  - [Type m-1] ({c} vs a) : les avantages (6342.0) >= le désavantage a (-160.0) => somme=6182.0
  - [Type m-1] ({e} vs b) : les avantages (4318.0) >= le désavantage b (-1408.0) => somme=2910.0
  - [Type m-1] ({f} vs d) : les avantages (4412.0) >= le désavantage d (-236.0) => somme=4176.0
  - [Type m-1] ({g} vs h) : les avantages (664.0) >= le désavantage h (-214.4) => somme=449.6
  - [Type m-1] ({j, m} vs i) : les avantages (3328.0) >= le désavantage i (-1550.0) => somme=1778.0
  - [Type m-1] ({n, o} vs k) : les avantages (2410.0) >= le désavantage k (-1616.0) => somme=794.0
  - [Type m-1] ({q} vs l) : les avantages (2120.0) >= le désavantage l (-566.0) => somme=1554.0
  - [Type m-1] ({r} vs p) : les avantages (4310.0) >= le désavantage p (-636.0) => somme=3674.0
  - [Type m-1] ({t} vs v) : les avantages (2702.0) >= le désavantage v (-1000.0) => somme

In [92]:
explication_mixte(weights, notes, criteres, "solution 2", "solution 4")

--- Recherche d'explication MIXTE (1-m) et (m-1) pour solution 2 > solution 4 ---

Une explication MIXTE existe. Groupes sélectionnés :

  - [Type 1-m] (h vs {r}) : l'avantage en h (4400.0) et les désavantages (-4126.0) => somme=274.0
  - [Type 1-m] (k vs {b, m, q, t, A}) : l'avantage en k (5056.0) et les désavantages (-5032.0) => somme=24.0
  - [Type 1-m] (w vs {c, f, o}) : l'avantage en w (4818.0) et les désavantages (-4622.0) => somme=196.0
  - [Type m-1] ({u, v} vs a) : les avantages (1198.0) >= le désavantage a (-1020.0) => somme=178.0
  - [Type m-1] ({s} vs d) : les avantages (900.0) >= le désavantage d (-608.0) => somme=292.0
  - [Type m-1] ({y} vs j) : les avantages (804.0) >= le désavantage j (-718.0) => somme=86.0
  - [Type m-1] ({e, z} vs l) : les avantages (4522.0) >= le désavantage l (-3922.0) => somme=600.0
  - [Type m-1] ({i} vs n) : les avantages (2602.0) >= le désavantage n (-2324.0) => somme=278.0
  - [Type m-1] ({g} vs p) : les avantages (924.0) >= le désavantage p (

In [93]:
explication_mixte(weights, notes, criteres, "solution 3", "solution 4")

--- Recherche d'explication MIXTE (1-m) et (m-1) pour solution 3 > solution 4 ---
Erreur: solution 3 n'est pas strictement meilleur que solution 4 au global.
